In [1]:
# imports
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.types import Float, Text, DateTime

In [2]:
# reading the CSV file

# location of the CSV sample file
csv = './pp-complete.csv'

# what each column in the CSV represents
header = ['id',
          'price',
          'transfer_date',
          'postcode',
          'type',
          'old_new',
          'duration',
          'paon',
          'saon',
          'street',
          'locality',
          'city',
          'district',
          'town',
          'category_type',
          'status']

df = pd.read_csv(csv,names=header)

In [3]:
# preparing the CSV to be loaded

# Filtering only relevant columns
pdd = df[['price','transfer_date','postcode','type','town']]

# Postcode should not be null
pdd = pdd[pdd.postcode.notnull()]

# Property type must be transformed: D -> DETACHED, S -> SEMI_DETACHED, T -> TERRACED, F -> FLATS, O -> OTHER
pdd['type'] = pdd['type'].replace(
    ['D','S','T','F','O'],
    ['DETACHED', 'SEMI_DETACHED', 'TERRACED', 'FLATS', 'OTHER'])


In [4]:
# connect to the local Postgres instance
engine = create_engine('postgresql://diego@localhost:5432/plentific_challenge_db')

# creating table using DataFrame
pdd.to_sql('pdd', 
            con=engine,
            index=False,
            if_exists='replace',
            chunksize=1000,
            dtype={
                'price':Float,
                'transfer_date':DateTime,
                'postcode':Text,
                'type':Text,
                'town':Text}
            )


In [5]:
# first report

# assuming these would input by the user:
from_date_given = '1995-03-01'
to_date_given = '1995-07-20'
postcode_given = 'S36 6UH'

query = engine.execute(
"""
    select type, date_trunc('month',transfer_date),avg(price)
    from pdd
    where postcode = '{postcode}'
        and transfer_date between '{from_date}' AND '{to_date}'
    group by 1,2
""".format(postcode = postcode_given,
           from_date = from_date_given,
           to_date = to_date_given)
)

result = pd.DataFrame(query.fetchall())
result.columns = query.keys()
print(result)


       type date_trunc      avg
0  DETACHED 1995-03-01  78450.0


In [6]:
# second report

# assuming these would input by the user:
postcode_given = 'S36 6UH'
year_given = 1995

query = engine.execute(
"""
    select price
    from pdd
    where postcode = '{postcode}'
        and date_part('year',transfer_date)={year}
""".format(postcode = postcode_given,
           year = year_given)
)

result = pd.DataFrame(query.fetchall())
result_consolidated = pd.cut(result.iloc[:,0],bins=8).value_counts()
print(result_consolidated)

(81625.0, 88250.0]      4
(74947.0, 81625.0]      3
(114750.0, 121375.0]    1
(121375.0, 128000.0]    1
(88250.0, 94875.0]      0
(94875.0, 101500.0]     0
(101500.0, 108125.0]    0
(108125.0, 114750.0]    0
Name: 0, dtype: int64
